# Pipeline for Microbiome Cross-Feeding Interactions

The following workflow gives in detail the steps that I used to generate the pipeline when running the script [`get_interactions.py`](https://github.com/ferreirav/CMEEResearchProject/blob/main/MiCRM_cross_feeding_project/code/get_interactions.py).
In sum, this script will subset the data from the Earth Microbiome Project by unique sample IDs, retrieve their organism composition, generate metabolic models and calculate the interaction potential.

### **Step 1:** Loading data...


I have loaded specifically 3 datasets:

- Samples data (filtered) from the EMP and used in [Machado et al. (2021)](https://www.pnas.org/doi/abs/10.1073/pnas.1421834112) - data can be directly downloaded [(here)](https://oc.embl.de/index.php/s/SbhoQa9YJoa748V/download) !

- Respective Metadata [(accesssed in EMP repo)](https://github.com/biocore/emp/tree/master/data/mapping-files)

- and the Genome-Scale Model list for bacterial species [(accessed here)](https://github.com/cdanielmachado/embl_gems)

The data was then merged by sample reference and extracted the specific features of interest.

A more detailed steps description of this data exploration is available in another notebook [(explore_data_v4.ipynb)](https://github.com/ferreirav/CMEEResearchProject/blob/main/MiCRM_cross_feeding_project/notebooks/exploring_data.ipynb) !

### Step 2: Creating Python script with functions

The script [`emp_data_wrang.py`](https://github.com/ferreirav/CMEEResearchProject/blob/main/MiCRM_cross_feeding_project/code/emp_data_wrang.py) contains three functions:

- **get_data()** which generates the data described in step 1;


- **download_ncbi_genome_temp()** which is a duplicate function of *download_ncbi_genome()* used in the `CarveMe` package but I have adapted it to download and decompress the `faa.gz` files according to sample being processed;

- **get_genome()** takes the organism list from the sample and download each single acession reference using the above function.

Also a CSV file with assembly accession and organism ID is created to be acessed later by a BASH script that will feed into the `carve` function

### Step 3: Calling `get_genome()`

Then `get_genome()` function is called once provided with the sample relevant information to download the whole community genomes and stores these in a folder with sample name which will be remove once completed the carving of metabolic models.

### Step 4: Calling BASH...

At this stage, I will be calling a Bash script to run `CarveMe` and `SMETANA`.

The BASH template for this step can be accessed [here](https://github.com/ferreirav/CMEEResearchProject/blob/main/MiCRM_cross_feeding_project/code/get_acessions.sh).

Initially, to capture the sample IDs and other data, I create a copy of this same template and named it as the sample ID.

To guide you through the bash script:

1. I collect the filename (same as sample ID), define the PATHS and create directories for each unique sample;

2. In a *for loop* I concatenate every line from the CSV file that contains accession number and organism ID, extract these and parse into the `CarveMe` command using:
        
        carve "acession".faa.gz -o "organism_name".xml    
    This will generate metabolic models for individual organisms and store these in an output folder with samples IDs.
<br>

3. Completed *carving*, the genomes folders and respective CSV file are deleted;

4. Collected the metabolic models, I use `SMETANA` to calculate community interaction and resource overlap using `global` options of the package:
            
        smetana -v *.xml -o "SAMPLE"     
    To calculate the individual and pair-wise interactions we need to use the `detailed` flag, as follows:
    
        smetana -v -d *.xml -o "SAMPLE"

#### <span style="color:red">**NOTE:**</span> These last steps, especially the calculation of individual interaction potential, is causing few errors, as:
- NAs in the *MIP* for community analysis:

     - **Solution** provided in [GitHub issue](https://github.com/cdanielmachado/smetana/issues/13) is to use the flag `--flavor bigg`. This calculates exchanges reactions using BiGG, otherwise searches for unbalanced reactions which include an internal "sink".
     
- 'MUS: Failed to find a minimal growth medium for ' + org_id:

    - **suggested** to use flag `--molweight` as minimises the minimal media composition predicted by minimising the total mass of the consumed substrates.

### Step 5: Multiprocessing

As last step on the script, I have implemented a process-based parallelism with `multiprocessing` module.

I call the function `get_data()` and retrieve only the sample list by using index `[0]`. I will then use `map()` to execute the main function `processing_interactions()` within the possible pool of CPUs available and giving as argument, for each run, an individual sample name from the `sample_list`.